In [1]:
from collections import Counter
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader, TensorDataset
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

from transformers import (
    DistilBertTokenizerFast, DistilBertForSequenceClassification,
    RobertaTokenizerFast, RobertaForSequenceClassification,
    XLNetTokenizer, XLNetForSequenceClassification,
    ElectraTokenizer, ElectraForSequenceClassification,
    DebertaTokenizer, DebertaForSequenceClassification
)
from collections import defaultdict
import numpy as np
import torch.nn.functional as F
import os
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import normalize
from scipy.stats import mode
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import warnings
from sklearn.exceptions import ConvergenceWarning
import os
warnings.filterwarnings('ignore', category=ConvergenceWarning)

In [ ]:
root = os.getcwd()
train_path = os.path.join(root, 'train.csv')
test_path = os.path.join(root, 'test.csv')
train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

In [5]:
similar_tokens = {}
with open("similar.txt", "r") as file:
    for line in file:
        tokens = line.strip().split(" ")
        similar_tokens[tokens[0].strip()] = [tokens[1].strip(), tokens[2].strip(), tokens[3].strip()]

In [8]:
def contains_token_1(text):
    tokens = text.split() if isinstance(text, str) else []
    return any(token in similar_tokens for token in tokens)

filtered_train_df = train_df[train_df['texts'].apply(contains_token_1)].reset_index(drop=True)

def augment_texts_with_similar_tokens(df, similar_tokens, replacement_index):
    augmented_texts = []
    augmented_labels = []

    for text, label in zip(df['texts'], df['labels']):
        tokens = text.split()  # Tokenize the text
        augmented_tokens = []

        for token in tokens:
            if token in similar_tokens:
                augmented_tokens.append(similar_tokens[token][replacement_index])
            else:
                augmented_tokens.append(token)  

        augmented_text = " ".join(augmented_tokens)
        augmented_texts.append(augmented_text)
        augmented_labels.append(label)

    return pd.DataFrame({'texts': augmented_texts, 'labels': augmented_labels})

df_1 = augment_texts_with_similar_tokens(filtered_train_df, similar_tokens, 0)  # Replace with token-2
df_2 = augment_texts_with_similar_tokens(filtered_train_df, similar_tokens, 1)  # Replace with token-3
df_3 = augment_texts_with_similar_tokens(filtered_train_df, similar_tokens, 2)  # Replace with token-4

In [12]:
augment_df = pd.concat([df_1, df_2, df_3], ignore_index=True)